In [1]:
import pandas as pd
import numpy as np
import datetime

# Daily Data by Health Board

The following Github Repo https://github.com/DataScienceScotland/COVID-19-Management-Information contains information regarding scottish information which is pulled on a daily basis. 

In [2]:
df_cases = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Cumulative%20cases.csv")
df_hospital_confirmed = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients%20-%20Confirmed.csv")
df_hospital_all = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients.csv")
df_icu = pd.read_csv("https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20ICU%20patients.csv")

In [3]:
df_icu

,Date,Ayrshire and Arran,Borders,Dumfries and Galloway,Fife,Forth Valley,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Western Isles,The Golden Jubilee National Hospital
0,2020-03-18,*,*,*,*,*,*,*,*,*,5,*,*,*,*,*
1,2020-03-19,*,*,*,*,*,*,*,*,*,5,*,*,*,*,*
2,2020-03-20,*,*,*,*,*,*,*,*,*,6,*,*,*,*,*
3,2020-03-21,*,*,*,*,*,*,*,*,*,9,*,*,*,*,*
4,2020-03-22,*,*,*,*,*,*,*,*,*,7,*,*,*,*,*
5,2020-03-23,*,*,*,*,*,*,6,*,*,7,*,*,5,*,*
6,2020-03-24,*,*,*,*,*,*,8,*,5,14,*,*,6,*,*
7,2020-03-25,*,*,*,*,*,*,11,*,8,14,*,*,6,*,*
8,2020-03-26,*,*,*,*,*,*,12,*,11,12,*,*,5,*,*
9,2020-03-27,7,*,*,*,*,*,18,*,9,15,*,*,6,*,*


In [4]:
list(df_icu.columns.values)

['Date',
 'Ayrshire and Arran',
 'Borders',
 'Dumfries and Galloway',
 'Fife',
 'Forth Valley',
 'Grampian',
 'Greater Glasgow and Clyde',
 'Highland',
 'Lanarkshire',
 'Lothian',
 'Orkney',
 'Shetland',
 'Tayside',
 'Western Isles',
 'The Golden Jubilee National Hospital']

In [5]:
path_dict = {
                "cases" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Cumulative%20cases.csv",
                "hospital_covid" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients%20-%20Confirmed.csv",
                "hospital_all" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20Hospital%20patients.csv",
                "icu" : "https://raw.githubusercontent.com/DataScienceScotland/COVID-19-Management-Information/master/COVID19%20-%20Daily%20Management%20Information%20-%20Scottish%20Health%20Boards%20-%20ICU%20patients.csv",
            }

In [6]:
def convert_date_to_week(date):
    date_list = date.split('-')
    date_num = list(map(int, date_list))
    a_date = datetime.date(date_num[0], date_num[1], date_num[2])
    week_number = a_date.isocalendar()[1]
    return week_number

In [7]:
def convert_date_to_day(date):
    date_list = date.split('-')
    date_num = list(map(int, date_list))
    a_date = datetime.date(date_num[0], date_num[1], date_num[2])
    day_of_year = (a_date - datetime.date(date_num[0], 1, 1)).days + 1
    return day_of_year

In [9]:
a = "2020-03-21"
convert_date_to_day(a)

81

In [25]:
def get_hbo_data(health_board, data_type):
    df = pd.read_csv(path_dict.get(data_type))
    df = df[['Date', health_board]]
    df['day_of_year'] = df['Date'].apply(lambda x : convert_date_to_day(x))
    df = df.set_index('day_of_year')
    df = df.drop(columns=['Date'])
    df = df.reset_index(inplace=False)
    return df

In [26]:
df_fife = get_hbo_data('Fife', 'hospital_all')

In [34]:
df_fife = df_fife.rename(columns={'day_of_year':'time', 'Fife':'valueA'})

In [35]:
df_fife.to_csv('fife_hospital_all2.csv')

In [38]:
df_fife

,time,valueA
0,78,18
1,79,24
2,80,20
3,81,16
4,82,23
5,83,28
6,84,42
7,85,28
8,86,45
9,87,51


In [50]:
df_fife['day_of_year'] = df_fife['Date'].apply(lambda x : convert_date_to_day(x))

In [53]:
df_fife

,Date,Fife,Date_new,day_of_year
0,2020-03-18,18,78,78
1,2020-03-19,24,79,79
2,2020-03-20,20,80,80
3,2020-03-21,16,81,81
4,2020-03-22,23,82,82
5,2020-03-23,28,83,83
6,2020-03-24,42,84,84
7,2020-03-25,28,85,85
8,2020-03-26,45,86,86
9,2020-03-27,51,87,87
